In [9]:
from tinydb import TinyDB, Query
import json
import os

# Manage DB.
class Reason:
    def __init__(self, db_name: str) -> None:
        self.db_instance = TinyDB(db_name)
        self.tables = {field: self.db_instance.table(field) for field in ["reality", "ideas", "metaphysics"]}

    def get_all(self, key: str, fields: list = []) -> list[dict]:
        search = self.tables[key].all()
        if fields:
            search = [{k: v for k, v in i.items() if k in fields} for i in search]
        return search
        

class Ontology(Reason):
    def __init__(self, db_name: str, custom_defaults_path: str = None) -> None:
        super().__init__(db_name)
        self.custom_defaults_path = custom_defaults_path
        self.DEFAULT = {
            "ideas": {
                "name": None,
                "id": None,
                "cat": "object",
                "desc": "An object.",
                "initRequire": [],
                "constRequire": [],
                "initGains": [],
                "constGains": [],
                "replace": [],
                "restrict": [],
                "visible": True,
                "turnToComplete": 0
            },
             "metaphysics": {
                "dismantleGain": 0.5,
                "turn": 1
            },
            "reality": {
                "id": None,
                "type": None,
                "amount": 1,
                "active": True
            }
        }

    # Set default values for partial entry.
    def set_default_values(self, obj: dict, defaults: dict) -> dict:
        for key in defaults.keys():
            if key not in obj.keys():
                obj.update({key: defaults[key]})
        return obj

    # Overwrite default values.
    def overwrite_defaults(self) -> None:
        try:
            with open(custom_defaults_path, "r") as f:
                self.DEFAULT = json.loads(f.read())
            
            print("Default values has been overwriten.")
        except OSError as e:
            raise Exception(e)

    # Check if data provided in scenrio is consistent.
    def check_integrity(self, content: dict, content_key: str) -> list[dict]:
        error_handle = []
        for element in content: 
            if len([el for el in element.keys() if el in ["name", "id"]]) == 2:
                content[content.index(element)] = self.set_default_values(element, content_key)
            else:
                error_handle.append(json.dumps(element))

        if not error_handle:
            return content
        else:
            err = '\n'.join(error_handle)
            print(f"No mandatory fields id and name in objects: indexes: {err}")

    # Compile data into database.
    def compile_data(self, scenario_dir: str) -> None:
        valid_files = [f for f in os.listdir(scenario_dir) if f.endswith(".json")]

        for file in valid_files:
            with open(f"{scenario_dir}/{file}", "r") as f:
                content = json.loads(f.read())
                for key in content.keys():
                    

In [11]:
13170+2186+5054+9902+1438+1168+508+487+12750+2360 = 49023

49023

In [12]:
49023 - 45944

3079